[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tiagodsilva/llm-programs/blob/main/03-self-consistency/03-self-consistency.ipynb)

In [ ]:
#@title Bootstrap (click to expand)

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab — installing dependencies...")
    
    # Fetch pyproject.toml
    !curl -L -o pyproject.toml https://raw.githubusercontent.com/tiagodsilva/llm-programs/main/pyproject.toml
    
    # Install poetry and dependencies
    %pip install uv
    !uv sync

In [ ]:
import sys

# Import the actual class from its new location
from llama_index.core import query_engine  # import the real module
from llama_index.experimental.query_engine import PandasQueryEngine

# Add PandasQueryEngine to the existing module
query_engine.PandasQueryEngine = PandasQueryEngine

# Register this fake module so downstream imports still work
sys.modules["llama_index.core.query_engine"] = query_engine

/Users/tiagodasilva/stuff/llm-programs/.venv/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


## Connect to the LLM provider

In [2]:
# Avoid unnecessary deprecation warnings
import shutup

shutup.please()

In [3]:
import os
import pandas as pd
from getpass import getpass


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass(var)


_set_if_undefined("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo-preview")

## Download the data set

In [5]:
# Silently download the data
dataset = "WikiTableQuestions"

if not os.path.exists(dataset):
    !curl -sL "https://github.com/ppasupat/WikiTableQuestions/releases/download/v1.0.2/WikiTableQuestions-1.0.2-compact.zip" -o data.zip
    !unzip -qq -o data.zip 

In [6]:
examples = pd.read_table(f"{dataset}/data/training-before300.tsv")

examples.head()

,id,utterance,context,targetValue
0,nt-0,what was the last year where this team was a p...,csv/204-csv/590.csv,2004
1,nt-1,in what city did piotr's last 1st place finish...,csv/204-csv/622.csv,"Bangkok, Thailand"
2,nt-2,which team won previous to crettyard?,csv/204-csv/772.csv,Wolfe Tones
3,nt-3,how many more passengers flew to los angeles t...,csv/203-csv/515.csv,"12,467"
4,nt-4,who was the opponent in the first game of the ...,csv/204-csv/495.csv,Derby County


In [7]:
example = examples.iloc[0]
question = example["utterance"]
context = example["context"]
print("The question is: ", question)

table = pd.read_csv("WikiTableQuestions/" + context)
table.head()

The question is:  what was the last year where this team was a part of the usl a-league?


,Year,Division,League,Regular Season,Playoffs,Open Cup,Avg. Attendance
0,2001,2,USL A-League,"4th, Western",Quarterfinals,Did not qualify,"7,169"
1,2002,2,USL A-League,"2nd, Pacific",1st Round,Did not qualify,"6,260"
2,2003,2,USL A-League,"3rd, Pacific",Did not qualify,Did not qualify,"5,871"
3,2004,2,USL A-League,"1st, Western",Quarterfinals,4th Round,"5,628"
4,2005,2,USL First Division,5th,Quarterfinals,4th Round,"6,028"


## Download Llama Index

In [8]:
from llama_index.llms.openai import OpenAI

llm = OpenAI()

In [9]:
from llama_index.packs.tables.mix_self_consistency.base import (
    MixSelfConsistencyQueryEngine,
)

## Textual reasoning only 

In [10]:
query_engine = MixSelfConsistencyQueryEngine(
    df=table, llm=llm, text_paths=1, symbolic_paths=0
)

In [11]:
response = await query_engine.aquery(example["utterance"])
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2003


In [12]:
query_engine = MixSelfConsistencyQueryEngine(
    df=table,
    llm=llm,
    text_paths=5,
    symbolic_paths=0,
    aggregation_mode="self-consistency",
    verbose=False,
)

In [ ]:
response = await query_engine.aquery(example["utterance"])
print(response)

2003


## Symbolic reasoning only

In [12]:
query_engine = MixSelfConsistencyQueryEngine(
    df=table,
    llm=llm,
    text_paths=0,
    symbolic_paths=1,
    aggregation_mode="none",
    verbose=True,
)

In [15]:
response = await query_engine.aquery(example["utterance"])
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2003


## Aggregation via Mix Self-Consistency

In [22]:
query_engine = MixSelfConsistencyQueryEngine(
    df=table,
    llm=llm,
    text_paths=4,
    symbolic_paths=5,
    aggregation_mode="self-consistency",
)

In [23]:
response = await query_engine.aquery(example["utterance"])
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/comp